In [4]:
import json
import requests
import torch
from PIL import Image
from io import BytesIO

# Load the YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

Using cache found in /Users/remaalnssiry/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-10-16 Python-3.10.6 torch-2.4.1 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [ ]:


# File paths
file_path = 'data/full_data.json'
output_file_path = 'data/indoor_outdoor_images.json'

# Load the JSON data from the file
with open(file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

# Define food and drink classes based on YOLOv5's dataset
food_classes = [
    'cake', 'pizza', 'hot dog', 'hamburger', 'fries', 'donut', 'spaghetti',
    'steak', 'soda', 'wine', 'coffee', 'tea', 'cocktail', 'watermelon',
    'fruit', 'vegetable'
]

# Initialize a list to hold images that do not contain food or drinks
filtered_images = []

def classify_image(url):
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    results = model(img)
    class_labels = results.names
    detected_classes = results.pred[0][:, -1].tolist()

    # Check if any of the detected classes are in the food_classes
    contains_food = any(class_labels[int(class_index)] in food_classes for class_index in detected_classes)

    return contains_food

# Loop through each item in the JSON data
for item in data:
    if 'photo_url' in item and item['photo_url']:
        if not classify_image(item['photo_url']):
            filtered_images.append(item)

# Save the filtered images to a new JSON file
with open(output_file_path, 'w', encoding='utf-8') as output_file:
    json.dump(filtered_images, output_file, ensure_ascii=False, indent=4)

print(f"Filtered images saved to '{output_file_path}'.")
